<a href="https://colab.research.google.com/github/Lafayola/HTML_Testing/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 65.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=22e5e8497c127796cf30ce7be712d3f9090c325e20c21906da0250b664d709b2
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [5]:

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [6]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [7]:

import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving export_dataframe.csv to export_dataframe.csv


In [11]:
import io
df = pd.read_csv(io.BytesIO(uploaded['export_dataframe.csv']))
df


,Unnamed: 0,index,title,text,subject,date,is_fake
0,7043.0,7043.0,Cubans fret over life after Castro with Trump ...,HAVANA (Reuters) - From the Bay of Pigs invasi...,politicsNews,27-Nov-16,0.0
1,33464.0,12047.0,OBAMA BRAGS About Hijacking 1.35 Million Acres...,President Barack Obama designated two national...,politics,28-Dec-16,1.0
2,11859.0,11859.0,Fighting in eastern Ukraine worst since Februa...,"KIEV/NOVOLUHANSKE, Ukraine (Reuters) - Fightin...",worldnews,20-Dec-17,0.0
3,40244.0,18827.0,"LIBERAL Lansing, MI Mayor Forced To REMOVE “Sa...",Watch his appearance on Tucker Carlson where h...,left-news,13-Apr-17,1.0
4,11827.0,11827.0,Philippine leader declares unilateral ceasefir...,MANILA (Reuters) - Philippine President Rodrig...,worldnews,20-Dec-17,0.0
...,...,...,...,...,...,...,...
44902,34507.0,13090.0,HAPPY LABOR DAY! 5 STUNNING FACTS: How Obama F...,Happy Labor Day! Today is a day to celebrate w...,politics,5-Sep-16,1.0
44903,7486.0,7486.0,Trump's chance of victory skyrockets on bettin...,NEW YORK (Reuters) - Online trading platform P...,politicsNews,8-Nov-16,0.0
44904,3020.0,3020.0,State Department will implement travel ban in ...,WASHINGTON (Reuters) - The State Department wi...,politicsNews,26-Jun-17,0.0
44905,30839.0,9422.0,WHOA! FEMINIST HACK LAWYER Gloria Allred REFUS...,"This story just keeps unraveling Two days ago,...",politics,15-Nov-17,1.0


In [13]:
# Data Cleaning
df=df.dropna()
df

,Unnamed: 0,index,title,text,subject,date,is_fake
0,7043.0,7043.0,Cubans fret over life after Castro with Trump ...,HAVANA (Reuters) - From the Bay of Pigs invasi...,politicsNews,27-Nov-16,0.0
1,33464.0,12047.0,OBAMA BRAGS About Hijacking 1.35 Million Acres...,President Barack Obama designated two national...,politics,28-Dec-16,1.0
2,11859.0,11859.0,Fighting in eastern Ukraine worst since Februa...,"KIEV/NOVOLUHANSKE, Ukraine (Reuters) - Fightin...",worldnews,20-Dec-17,0.0
3,40244.0,18827.0,"LIBERAL Lansing, MI Mayor Forced To REMOVE “Sa...",Watch his appearance on Tucker Carlson where h...,left-news,13-Apr-17,1.0
4,11827.0,11827.0,Philippine leader declares unilateral ceasefir...,MANILA (Reuters) - Philippine President Rodrig...,worldnews,20-Dec-17,0.0
...,...,...,...,...,...,...,...
44902,34507.0,13090.0,HAPPY LABOR DAY! 5 STUNNING FACTS: How Obama F...,Happy Labor Day! Today is a day to celebrate w...,politics,5-Sep-16,1.0
44903,7486.0,7486.0,Trump's chance of victory skyrockets on bettin...,NEW YORK (Reuters) - Online trading platform P...,politicsNews,8-Nov-16,0.0
44904,3020.0,3020.0,State Department will implement travel ban in ...,WASHINGTON (Reuters) - The State Department wi...,politicsNews,26-Jun-17,0.0
44905,30839.0,9422.0,WHOA! FEMINIST HACK LAWYER Gloria Allred REFUS...,"This story just keeps unraveling Two days ago,...",politics,15-Nov-17,1.0


In [14]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="export_dataframe.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("export_dataframe.csv"), sep=",", header=True)

# Show DataFrame
df.show(truncate=False)

+-----+-----+----------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# Tokenize DataFrame
review_data = Tokenizer(inputCol="text", outputCol="Words")
review_data

Tokenizer_ac36fd31e846

In [16]:

# Transform DataFrame
reviewed = review_data.transform(df)
reviewed.show(truncate=False)

+-----+-----+----------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:

# Remove stop words
remover = StopWordsRemover(inputCol="Words", outputCol="filtered")
remover

StopWordsRemover_27a1018b6099

In [18]:
# Transform new DataFrame
newFrame = remover.transform(reviewed)
newFrame.show(truncate=False)

+-----+-----+----------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
# Show simplified review
newFrame.select("filtered").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
newFrame.show()

+-----+-----+--------------------+--------------------+------------+---------+-------+--------------------+--------------------+
|  _c0|index|               title|                text|     subject|     date|is_fake|               Words|            filtered|
+-----+-----+--------------------+--------------------+------------+---------+-------+--------------------+--------------------+
| 7043| 7043|Cubans fret over ...|HAVANA (Reuters) ...|politicsNews|27-Nov-16|      0|[havana, (reuters...|[havana, (reuters...|
|33464|12047|OBAMA BRAGS About...|President Barack ...|    politics|28-Dec-16|      1|[president, barac...|[president, barac...|
|11859|11859|Fighting in easte...|KIEV/NOVOLUHANSKE...|   worldnews|20-Dec-17|      0|[kiev/novoluhansk...|[kiev/novoluhansk...|
|40244|18827|LIBERAL Lansing, ...|Watch his appeara...|   left-news|13-Apr-17|      1|[watch, his, appe...|[watch, appearanc...|
|11827|11827|Philippine leader...|MANILA (Reuters) ...|   worldnews|20-Dec-17|      0|[manila, (r

In [21]:
newdf = newFrame.drop('Words','text')
newdf

DataFrame[_c0: string, index: string, title: string, subject: string, date: string, is_fake: string, filtered: array<string>]

In [22]:
newdf.show(truncate=False)

+-----+-----+----------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- index: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- is_fake: string (nullable = true)



In [24]:
X = newdf.drop("label", axis=1)
y = newdf["label"]
print(X.shape, y.shape)

TypeError: ignored

In [ ]:
# Dividing the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

newdf = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
newdf.head()